# LOAD DATA

In [1]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

# Download both 'punkt' and 'punkt_tab' tokenizer data
nltk.download('punkt')      # For compatibility with older NLTK versions
nltk.download('punkt_tab')  # Required for newer NLTK versions

# Load dataset with tab-separated values
file_path = r"/content/spoc-train.tsv"  # Change to your dataset's path

# Use sep='\t' to correctly read TSV files
df = pd.read_csv(file_path, sep='\t', on_bad_lines='skip')

# Display dataset information
print(df.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


                                              text  \
0                in the function gcd(a,b=integers)   
1  if b=1 return a, else call function gcd(b, a%b)   
2                                              NaN   
3                                              NaN   
4               n , nn, ans = integers with ans =0   

                             code  workerid probid       subid  line  indent  
0         int gcd(int a, int b) {        38    13A  41120785.0   0.0     0.0  
1  return !b ? a : gcd(b, a % b);        38    13A  41120785.0   1.0     1.0  
2                               }        38    13A  41120785.0   2.0     0.0  
3                    int main() {        38    13A  41120785.0   3.0     0.0  
4             int n, nn, ans = 0;        38    13A  41120785.0   4.0     1.0  


In [2]:
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109002 entries, 0 to 109001
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   text      80289 non-null   object 
 1   code      109002 non-null  object 
 2   workerid  109002 non-null  int64  
 3   probid    109002 non-null  object 
 4   subid     109001 non-null  float64
 5   line      109001 non-null  float64
 6   indent    109001 non-null  float64
dtypes: float64(3), int64(1), object(3)
memory usage: 5.8+ MB
None


# DATA PREPROCESSING

In [3]:
# Convert 'text' column to string and tokenize
df["text"] = df["text"].astype(str)
df["text_tokens"] = df["text"].apply(word_tokenize)

In [4]:
df["text"] = df["text"].fillna("")  # Replace NaN with empty strings
df["text_tokens"] = df["text"].apply(word_tokenize)


In [5]:
df["text_tokens"] = df["text"].apply(lambda x: word_tokenize(x) if isinstance(x, str) else [])


In [6]:
print(df["text"].dtype)  # Check the column's data type
print(df["text"].isna().sum())  # Count missing values
print(df[df["text"].apply(lambda x: not isinstance(x, str))])  # Show non-string values


object
0
Empty DataFrame
Columns: [text, code, workerid, probid, subid, line, indent, text_tokens]
Index: []


# TOKENIZATION

In [7]:
import nltk
from nltk.tokenize import word_tokenize

# Download tokenizer if not available
nltk.download('punkt')

# Tokenizing pseudocode and code
df["text_tokens"] = df["text"].apply(word_tokenize)
df["code_tokens"] = df["code"].apply(word_tokenize)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# PRINT TOKEN SAMPLES

In [8]:
# Print samples from index 1-10
print("Samples from index 1-10:")
for i in range(1, 11):
    print(f"Index {i}:")
    print("Tokenized Pseudocode:", df["text_tokens"].iloc[i])
    print("Tokenized C++ Code:", df["code_tokens"].iloc[i])
    print("-" * 50)

# Print samples from index 20-30
print("\nSamples from index 20-30:")
for i in range(20, 31):
    print(f"Index {i}:")
    print("Tokenized Pseudocode:", df["text_tokens"].iloc[i])
    print("Tokenized C++ Code:", df["code_tokens"].iloc[i])
    print("-" * 50)


Samples from index 1-10:
Index 1:
Tokenized Pseudocode: ['if', 'b=1', 'return', 'a', ',', 'else', 'call', 'function', 'gcd', '(', 'b', ',', 'a', '%', 'b', ')']
Tokenized C++ Code: ['return', '!', 'b', '?', 'a', ':', 'gcd', '(', 'b', ',', 'a', '%', 'b', ')', ';']
--------------------------------------------------
Index 2:
Tokenized Pseudocode: ['nan']
Tokenized C++ Code: ['}']
--------------------------------------------------
Index 3:
Tokenized Pseudocode: ['nan']
Tokenized C++ Code: ['int', 'main', '(', ')', '{']
--------------------------------------------------
Index 4:
Tokenized Pseudocode: ['n', ',', 'nn', ',', 'ans', '=', 'integers', 'with', 'ans', '=0']
Tokenized C++ Code: ['int', 'n', ',', 'nn', ',', 'ans', '=', '0', ';']
--------------------------------------------------
Index 5:
Tokenized Pseudocode: ['Read', 'n']
Tokenized C++ Code: ['cin', '>', '>', 'n', ';']
--------------------------------------------------
Index 6:
Tokenized Pseudocode: ['for', 'i=2', 'to', 'n-1', 'execu

In [9]:
# Save tokenized pseudocode and C++ code to CSV
output_file = "tokenized_spoc.csv"
df[["text_tokens", "code_tokens"]].to_csv(output_file, index=False)

print(f"Tokenized data saved to {output_file}")


Tokenized data saved to tokenized_spoc.csv


In [10]:
# Add start and end tokens to tokenized C++ code
df["code_tokens"] = df["code_tokens"].apply(lambda tokens: ["<start>"] + tokens + ["<end>"])

# Save updated tokenized data to CSV
output_file = "tokenized_spoc_with_tokens.csv"
df[["text_tokens", "code_tokens"]].to_csv(output_file, index=False)

print(f"Updated tokenized data saved to {output_file}")


Updated tokenized data saved to tokenized_spoc_with_tokens.csv


In [11]:
# Make "text_tokens" and "code_tokens" length same by padding with "<pad>"
max_len = max(df["text_tokens"].apply(len).max(), df["code_tokens"].apply(len).max())

df["text_tokens"] = df["text_tokens"].apply(lambda tokens: tokens + ["<pad>"] * (max_len - len(tokens)))
df["code_tokens"] = df["code_tokens"].apply(lambda tokens: tokens + ["<pad>"] * (max_len - len(tokens)))

# Save padded tokenized data to CSV
output_file = "tokenized_spoc_padded.csv"
df[["text_tokens", "code_tokens"]].to_csv(output_file, index=False)

print(f"Padded tokenized data saved to {output_file}")


Padded tokenized data saved to tokenized_spoc_padded.csv


In [12]:
import json

# Define special tokens with fixed indices
vocab = {
    "<unk>": 0,
    "<pad>": 1,
    "<start>": 2,
    "<end>": 3
}

# Assign indices to other tokens
for column in ["text_tokens", "code_tokens"]:
    for tokens in df[column]:
        for token in tokens:
            if token not in vocab:
                vocab[token] = len(vocab)

# Save vocabulary to JSON
vocab_file = "vocabulary.json"
with open(vocab_file, "w") as f:
    json.dump(vocab, f, indent=4)

print(f"Vocabulary saved to {vocab_file}")


Vocabulary saved to vocabulary.json


In [13]:
# Load vocabulary
with open("vocabulary.json", "r") as f:
    vocab = json.load(f)

# Load tokenized data
df = pd.read_csv("/content/tokenized_spoc_padded.csv")

# Convert string tokens to lists
df["text_tokens"] = df["text_tokens"].apply(eval)
df["code_tokens"] = df["code_tokens"].apply(eval)

# Convert tokens to sequences using vocabulary
df["text_sequences"] = df["text_tokens"].apply(lambda tokens: [vocab.get(token, vocab["<unk>"]) for token in tokens])
df["code_sequences"] = df["code_tokens"].apply(lambda tokens: [vocab.get(token, vocab["<unk>"]) for token in tokens])

# Save sequences to CSV
output_file = "tokenized_sequences.csv"
df[["text_sequences", "code_sequences"]].to_csv(output_file, index=False)

print(f"Tokenized sequences saved to {output_file}")


Tokenized sequences saved to tokenized_sequences.csv


In [14]:
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import torch
import ast
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm

class DataLoad(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.inputs = [ast.literal_eval(x) for x in df['text_sequences']]
        self.outputs = [ast.literal_eval(x) for x in df['code_sequences']]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_tensor = torch.tensor(self.inputs[idx], dtype=torch.int64)
        output_tensor = torch.tensor(self.outputs[idx], dtype=torch.int64)
        return input_tensor, output_tensor

def Add_Pad(batch):
    inputs, outputs = zip(*batch)
    inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
    outputs = pad_sequence(outputs, batch_first=True, padding_value=0)
    return inputs, outputs

# Load dataset and dataloader
dataset = DataLoad('/content/tokenized_sequences.csv')
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=Add_Pad)

# Iterate with progress bar
data_iter = iter(dataloader)
for batch in tqdm(dataloader, desc="Loading Batches"):
    features, labels = batch  # Get a batch of data
    break  # Remove this if you want to iterate over all batches

print("Batch loaded successfully!")


Loading Batches:   0%|          | 0/1704 [00:00<?, ?it/s]

Batch loaded successfully!


In [15]:
print(features)
print(labels)

tensor([[ 21,  24, 235,  ...,   1,   1,   1],
        [ 34,  88,  68,  ...,   1,   1,   1],
        [ 34,  23,  31,  ...,   1,   1,   1],
        ...,
        [102,  55,  41,  ...,   1,   1,   1],
        [ 20,   1,   1,  ...,   1,   1,   1],
        [ 16,  13, 159,  ...,   1,   1,   1]])
tensor([[  2, 235,  21,  ...,   1,   1,   1],
        [  2, 266,  24,  ...,   1,   1,   1],
        [  2,  23,  24,  ...,   1,   1,   1],
        ...,
        [  2, 235,  41,  ...,   1,   1,   1],
        [  2, 498,   3,  ...,   1,   1,   1],
        [  2,  16,  13,  ...,   1,   1,   1]])


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import math

# Transformer Hyperparameters
class Config:
    vocab_size = 12006  # Adjust based on vocabulary.json
    max_length = 100  # Adjust based on sequence length
    embed_dim = 256
    num_heads = 8
    num_layers =2
    feedforward_dim = 512
    dropout = 0.1
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = Config()

# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=100):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)  # Shape: (1, max_len, embed_dim)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

# Transformer Model
class PseudoCodeTransformer(nn.Module):
    def __init__(self, config):
        super(PseudoCodeTransformer, self).__init__()
        self.embedding = nn.Embedding(config.vocab_size, config.embed_dim)
        self.positional_encoding = PositionalEncoding(config.embed_dim, config.max_length)

        self.transformer = nn.Transformer(
            d_model=config.embed_dim,
            nhead=config.num_heads,
            num_encoder_layers=config.num_layers,
            num_decoder_layers=config.num_layers,
            dim_feedforward=config.feedforward_dim,
            dropout=config.dropout
        )

        self.fc_out = nn.Linear(config.embed_dim, config.vocab_size)
        self.dropout = nn.Dropout(config.dropout)

    def generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1).to(config.device)

    def forward(self, src, tgt):
        src_emb = self.embedding(src) * math.sqrt(config.embed_dim)
        tgt_emb = self.embedding(tgt) * math.sqrt(config.embed_dim)

        src_emb = self.positional_encoding(src_emb)
        tgt_emb = self.positional_encoding(tgt_emb)

        src_mask = self.generate_square_subsequent_mask(src.size(1))
        tgt_mask = self.generate_square_subsequent_mask(tgt.size(1))

        out = self.transformer(src_emb.permute(1, 0, 2), tgt_emb.permute(1, 0, 2),
                               src_mask=src_mask, tgt_mask=tgt_mask)

        out = self.fc_out(out.permute(1, 0, 2))  # Convert back to batch-first
        return out

# Initialize Model
model = PseudoCodeTransformer(config).to(config.device)
print("Model initialized successfully!")


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Model initialized successfully!


In [17]:
def translate(model, pseudocode_tokens, vocab, device, max_length=50):
    model.eval()

    # Convert pseudocode tokens to numerical indices
    input_ids = [vocab.get(token, vocab["<unk>"]) for token in pseudocode_tokens]
    input_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to(device)  # Add batch dimension

    # Start token for generation
    output_ids = [vocab["<start>"]]

    for _ in range(max_length):
        output_tensor = torch.tensor(output_ids, dtype=torch.long).unsqueeze(0).to(device)

        # Get model predictions
        with torch.no_grad():
            predictions = model(input_tensor, output_tensor)

        # Select the most probable token
        next_token_id = predictions.argmax(dim=-1)[:, -1].item()
        output_ids.append(next_token_id)

        # Stop if end token is generated
        if next_token_id == vocab["<end>"]:
            break

    # Convert token indices back to words
    id_to_token = {idx: token for token, idx in vocab.items()}
    translated_code = [id_to_token.get(idx, "<unk>") for idx in output_ids[1:]]  # Exclude <start> token

    return " ".join(translated_code)


In [18]:
import json

# Load vocabulary
with open("vocabulary.json", "r") as f:
    vocab = json.load(f)

# Ensure vocab is a dictionary
print(f"✅ Vocabulary loaded with {len(vocab)} tokens")


✅ Vocabulary loaded with 6656 tokens


In [19]:
from torch.utils.data import DataLoader
import torch.nn.functional as F
from tqdm import tqdm
import os

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔹 Using device: {device}")

# Move model to device
model.to(device)

# Loss Function & Optimizer
criterion = nn.CrossEntropyLoss(ignore_index=1)  # Ignore padding token
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-3)

# Create directory to save models
os.makedirs("checkpoints", exist_ok=True)

# Training Loop
num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch in progress_bar:
        src, tgt = batch
        src, tgt = src.to(device), tgt.to(device)  # Move batch to GPU

        tgt_input = tgt[:, :-1]  # Remove <end> token
        tgt_output = tgt[:, 1:]  # Shifted version

        optimizer.zero_grad()
        output = model(src, tgt_input)

        loss = criterion(output.view(-1, config.vocab_size), tgt_output.contiguous().view(-1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # Save Model Checkpoint
    torch.save(model.state_dict(), f"checkpoints/transformer_epoch_{epoch+1}.pth")
    print(f"✅ Model saved: checkpoints/transformer_epoch_{epoch+1}.pth")

    # Print Example Prediction
    model.eval()
    example_pseudocode = ["create", "integer", "x"]
    translated_code = translate(model, example_pseudocode, vocab, device)
    print(f"🔹 Example Prediction (Pseudocode → C++): {translated_code}\n")


🔹 Using device: cuda


Epoch 1/1: 100%|██████████| 1704/1704 [02:42<00:00, 10.49it/s, loss=0.361]


Epoch [1/1], Loss: 1.2264
✅ Model saved: checkpoints/transformer_epoch_1.pth
🔹 Example Prediction (Pseudocode → C++): int x x x x x = int x x x x x , int x = int x x x ; <end>



In [20]:
example_pseudocode = ["for", "i", "=", "0", "to", "size", "of", "ans", "exclusive", ",", "print", "ans", "[", "i", "]", "print", "newline"]
translated_code = translate(model, example_pseudocode, vocab, device)
print(f"🔹 Example Prediction (Pseudocode → C++): {translated_code}\n")

🔹 Example Prediction (Pseudocode → C++): for ( int i = 0 ; i < ans = 0 ; i++ ) { ans for ( ans = abs ( ans ( ans ( ans , ans ( ans ) ; } <end>



In [26]:
for (int i = 0; i < ans.size(); i++) { cout << ans[i] << endl; }

SyntaxError: invalid syntax. Perhaps you forgot a comma? (<ipython-input-26-ee7d37ebdb44>, line 1)

In [28]:
# Load the trained model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔹 Using device: {device}")

# Move model to device
model.to(device)

model = PseudoCodeTransformer(config).to(device)
model.load_state_dict(torch.load("/content/checkpoints/transformer_epoch_1.pth", map_location=device))
model.eval()

# Run translation on example pseudocode
example_pseudocode = ["for", "i", "=", "0", "to", "size", "of", "ans", "exclusive", ",", "print", "ans", "[", "i", "]", "print", "newline"]
translated_code = translate(model, example_pseudocode, vocab, device)

print(f"🔹 Example Prediction (Pseudocode → C++): {translated_code}\n")


🔹 Using device: cuda
🔹 Example Prediction (Pseudocode → C++): for ( int i = 0 ; i < ans = 0 ; i++ ) { ans for ( ans = abs ( ans ( ans ( ans , ans ( ans ) ; } <end>



<ipython-input-28-6b8e2ac4d067>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/checkpoints/transformer_epoch_1.pth", map_locati

**C++ CODE TO PSEUDOCODE**

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import math
import pandas as pd
import ast
import json
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import os

# Transformer Hyperparameters
class Config:
    vocab_size = 12006  # Adjust based on vocabulary.json
    max_length = 100
    embed_dim = 256
    num_heads = 8
    num_layers = 2
    feedforward_dim = 512
    dropout = 0.1
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config = Config()

# Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_len=100):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, embed_dim)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2).float() * (-math.log(10000.0) / embed_dim))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.pe = pe.unsqueeze(0)

    def forward(self, x):
        return x + self.pe[:, :x.size(1)].to(x.device)

# Transformer Model
class CPPtoPseudoTransformer(nn.Module):
    def __init__(self, config):
        super(CPPtoPseudoTransformer, self).__init__()
        self.embedding = nn.Embedding(config.vocab_size, config.embed_dim)
        self.positional_encoding = PositionalEncoding(config.embed_dim, config.max_length)

        self.transformer = nn.Transformer(
            d_model=config.embed_dim,
            nhead=config.num_heads,
            num_encoder_layers=config.num_layers,
            num_decoder_layers=config.num_layers,
            dim_feedforward=config.feedforward_dim,
            dropout=config.dropout
        )

        self.fc_out = nn.Linear(config.embed_dim, config.vocab_size)
        self.dropout = nn.Dropout(config.dropout)

    def generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1).to(config.device)

    def forward(self, src, tgt):
        src_emb = self.embedding(src) * math.sqrt(config.embed_dim)
        tgt_emb = self.embedding(tgt) * math.sqrt(config.embed_dim)

        src_emb = self.positional_encoding(src_emb)
        tgt_emb = self.positional_encoding(tgt_emb)

        src_mask = self.generate_square_subsequent_mask(src.size(1))
        tgt_mask = self.generate_square_subsequent_mask(tgt.size(1))

        out = self.transformer(src_emb.permute(1, 0, 2), tgt_emb.permute(1, 0, 2),
                               src_mask=src_mask, tgt_mask=tgt_mask)

        out = self.fc_out(out.permute(1, 0, 2))
        return out

# Initialize Model
model = CPPtoPseudoTransformer(config).to(config.device)
print("🚀 C++ → Pseudocode Model initialized!")


🚀 C++ → Pseudocode Model initialized!


In [30]:

# Load Vocabulary
with open("vocabulary.json", "r") as f:
    vocab = json.load(f)

print(f"✅ Vocabulary loaded with {len(vocab)} tokens")



✅ Vocabulary loaded with 6656 tokens


In [31]:
def translate2(model, cpp_tokens, vocab, device, max_length=50):
    model.eval()

    # Convert C++ tokens to numerical indices
    input_ids = [vocab.get(token, vocab["<unk>"]) for token in cpp_tokens]
    input_tensor = torch.tensor(input_ids, dtype=torch.long).unsqueeze(0).to(device)  # Add batch dimension

    output_ids = [vocab["<start>"]]

    for _ in range(max_length):
        output_tensor = torch.tensor(output_ids, dtype=torch.long).unsqueeze(0).to(device)

        # Get model predictions
        with torch.no_grad():
            predictions = model(input_tensor, output_tensor)

        # Select the most probable token
        next_token_id = predictions.argmax(dim=-1)[:, -1].item()

        if next_token_id == vocab["<pad>"]:  # Ignore <pad> tokens
            continue

        output_ids.append(next_token_id)

        if next_token_id == vocab["<end>"]:  # Stop if <end> is generated
            break

    # Convert token indices back to words
    id_to_token = {idx: token for token, idx in vocab.items()}
    translated_pseudocode = [id_to_token.get(idx, "<unk>") for idx in output_ids[1:]]  # Exclude <start>

    return " ".join(translated_pseudocode)


In [33]:
# Dataset Class
class CPPToPseudoDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.inputs = [ast.literal_eval(x) for x in df['code_sequences']]
        self.outputs = [ast.literal_eval(x) for x in df['text_sequences']]

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_tensor = torch.tensor(self.inputs[idx], dtype=torch.int64)
        output_tensor = torch.tensor([vocab["<start>"]] + self.outputs[idx] + [vocab["<end>"]], dtype=torch.int64)
        return input_tensor, output_tensor

# Padding Function
def Add_Pad(batch):
    inputs, outputs = zip(*batch)
    inputs = pad_sequence(inputs, batch_first=True, padding_value=0)
    outputs = pad_sequence(outputs, batch_first=True, padding_value=0)
    return inputs, outputs

# Load Dataset
dataset = CPPToPseudoDataset("tokenized_sequences.csv")
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, collate_fn=Add_Pad)

print(f"✅ Loaded {len(dataset)} examples for training")

# Training Configuration
criterion = nn.CrossEntropyLoss(ignore_index=vocab["<pad>"])
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-3)

# Create directory to save models
os.makedirs("checkpoints", exist_ok=True)


✅ Loaded 109002 examples for training


In [36]:
# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for batch in progress_bar:
        src, tgt = batch
        src, tgt = src.to(config.device), tgt.to(config.device)

        tgt_input = tgt[:, :-1]
        tgt_output = tgt[:, 1:]

        optimizer.zero_grad()
        output = model(src, tgt_input)

        loss = criterion(output.view(-1, config.vocab_size), tgt_output.contiguous().view(-1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix(loss=loss.item())

    avg_loss = epoch_loss / len(dataloader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

    # Save Model Checkpoint
    torch.save(model.state_dict(), f"checkpoints/cpp_to_pseudo_epoch_{epoch+1}.pth")
    print(f"✅ Model saved: checkpoints/cpp_to_pseudo_epoch_{epoch+1}.pth")

    # Print Example Prediction
    model.eval()
    example_cpp = ["int", "main", "(", ")", "{", "return", "0", ";", "}"]
    translated_pseudocode = translate2(model, example_cpp, vocab, config.device)
    print(f"🔹 Example Prediction (C++ → Pseudocode): {translated_pseudocode}\n")


Epoch 1/5: 100%|██████████| 1704/1704 [02:41<00:00, 10.57it/s, loss=1.33]


Epoch [1/5], Loss: 1.0850
✅ Model saved: checkpoints/cpp_to_pseudo_epoch_1.pth
🔹 Example Prediction (C++ → Pseudocode): declare integer called with integer arguments , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return



Epoch 2/5: 100%|██████████| 1704/1704 [02:43<00:00, 10.43it/s, loss=1.14]


Epoch [2/5], Loss: 1.0870
✅ Model saved: checkpoints/cpp_to_pseudo_epoch_2.pth
🔹 Example Prediction (C++ → Pseudocode): declare integer called with integer arguments , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return



Epoch 3/5: 100%|██████████| 1704/1704 [02:43<00:00, 10.43it/s, loss=1.88]


Epoch [3/5], Loss: 1.0857
✅ Model saved: checkpoints/cpp_to_pseudo_epoch_3.pth
🔹 Example Prediction (C++ → Pseudocode): declare integer called with integer arguments , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return



Epoch 4/5: 100%|██████████| 1704/1704 [02:43<00:00, 10.42it/s, loss=0.686]


Epoch [4/5], Loss: 1.0851
✅ Model saved: checkpoints/cpp_to_pseudo_epoch_4.pth
🔹 Example Prediction (C++ → Pseudocode): declare integer called with integer arguments , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return



Epoch 5/5: 100%|██████████| 1704/1704 [02:43<00:00, 10.43it/s, loss=1.16]


Epoch [5/5], Loss: 1.0853
✅ Model saved: checkpoints/cpp_to_pseudo_epoch_5.pth
🔹 Example Prediction (C++ → Pseudocode): declare integer called with integer arguments , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return 0 , return



In [37]:
# Load the trained model

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🔹 Using device: {device}")

# Move model to device
model.to(device)

model = PseudoCodeTransformer(config).to(device)
model.load_state_dict(torch.load("/content/checkpoints/cpp_to_pseudo_epoch_4.pth", map_location=device))
model.eval()

example_cpp = ["int", "a","=", "10",";"]
translated_pseudocode = translate2(model, example_cpp, vocab, config.device)
print(f"🔹 Example Prediction (C++ → Pseudocode): {translated_pseudocode}\n")


🔹 Using device: cuda


<ipython-input-37-62b96cae03d2>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/content/checkpoints/cpp_to_pseudo_epoch_4.pth", map_loca

🔹 Example Prediction (C++ → Pseudocode): create integer a integer a with a = 10 = 10 = 10 = 10 = 10 = 10 10 = 10 = 10 10 = 10 10 = 10 = 10 = 10 = 10 = 10 10 = 10 = 10 10 = 10 10 = 10 =

